In [43]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import dask.dataframe as dd
from dask.distributed import Client

In [44]:
client = Client(n_workers=16)
client

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/distributed/bokeh/core.py:57: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn('\n' + msg)


Client Scheduler: tcp://127.0.0.1:41091 Dashboard: http://127.0.0.1:46633/status,Cluster Workers: 16 Cores: 16 Memory: 67.53 GB


In [45]:
!pwd

/home/ec2-user/SageMaker/DS-Unit-3-Sprint-3-Big-Data/module1-aws-sagemaker


In [46]:
data_loc = 'instacart_2017_05_01/'

In [47]:
aisles = dd.read_csv(data_loc + 'aisles.csv', header=0, enforce=True)
aisles.head()

,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation


In [48]:
depts = dd.read_csv(data_loc + 'departments.csv', header=0, enforce=True)
depts.head()

,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol


In [49]:
order_products = dd.read_csv(data_loc + 'order_products*.csv', header=0)
order_products.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 4 entries, order_id to reordered
dtypes: int64(4)

In [50]:
order_products.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [51]:
products = dd.read_csv(data_loc + 'products.csv')
products.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 4 entries, product_id to department_id
dtypes: object(1), int64(3)

In [52]:
products.head()

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [53]:
order_products = dd.merge(products, order_products)
order_products.head()

,product_id,product_name,aisle_id,department_id,order_id,add_to_cart_order,reordered
0,1,Chocolate Sandwich Cookies,61,19,1107,7,0
1,1,Chocolate Sandwich Cookies,61,19,5319,3,1
2,1,Chocolate Sandwich Cookies,61,19,7540,4,1
3,1,Chocolate Sandwich Cookies,61,19,9228,2,0
4,1,Chocolate Sandwich Cookies,61,19,9273,30,0


In [54]:
order_products = order_products.persist()

In [55]:
order_products_counts = order_products['product_name'].value_counts().compute()

In [56]:
order_products_counts[:100]

Banana                                                            491291
Bag of Organic Bananas                                            394930
Organic Strawberries                                              275577
Organic Baby Spinach                                              251705
Organic Hass Avocado                                              220877
Organic Avocado                                                   184224
Large Lemon                                                       160792
Strawberries                                                      149445
Limes                                                             146660
Organic Whole Milk                                                142813
Organic Raspberries                                               142603
Organic Yellow Onion                                              117716
Organic Garlic                                                    113936
Organic Zucchini                                   

In [57]:
depts.head(100)

,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol
5,6,international
6,7,beverages
7,8,pets
8,9,dry goods pasta
9,10,bulk


In [58]:
aisles.head(3000)

,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation
5,6,other
6,7,packaged meat
7,8,bakery desserts
8,9,pasta sauce
9,10,kitchen supplies


In [59]:
%%time
grouped = order_products.groupby(['department_id', 'aisle_id'])[['reordered']] \
            .apply(lambda x: x.nlargest(2, columns=['reordered']), \
                  meta={'reordered': 'int64'}).compute()

CPU times: user 3.39 s, sys: 774 ms, total: 4.17 s
Wall time: 12.7 s


In [60]:
grouped = grouped.reset_index().rename(columns={'level_2': 'counts'})
grouped = grouped.sort_values(by='counts', ascending=False)

In [96]:
grouped['%'] = grouped['counts'] / sum(grouped['counts']) * 100
grouped['cumulative %'] = grouped['%'].cumsum()
grouped

,department_id,aisle_id,counts,reordered,%,cumulative %
247,18,102,196705,1,2.894599,2.894599
246,18,102,196700,1,2.894525,5.789124
117,4,32,183162,1,2.695308,8.484431
116,4,32,183161,1,2.695293,11.179724
93,6,76,145002,1,2.133767,13.313491
45,11,118,133093,1,1.958521,15.272012
92,6,76,129683,1,1.908341,17.180353
191,11,44,128588,1,1.892228,19.072581
195,12,35,116026,1,1.707372,20.779953
194,12,35,116025,1,1.707358,22.487311


In [97]:
grouped_dd = dd.from_pandas(grouped, npartitions=1)

In [98]:
grouped_dd = grouped_dd.merge(aisles, left_on='aisle_id', right_on='aisle_id')
grouped_dd = grouped_dd.nlargest(n=3000, columns='counts')

In [99]:
grouped_dd.head(10)

,department_id,aisle_id,counts,reordered,%,cumulative %,aisle
247,18,102,196705,1,2.894599,2.894599,baby bath body care
246,18,102,196700,1,2.894525,5.789124,baby bath body care
117,4,32,183162,1,2.695308,8.484431,packaged produce
116,4,32,183161,1,2.695293,11.179724,packaged produce
93,6,76,145002,1,2.133767,13.313491,indian foods
45,11,118,133093,1,1.958521,15.272012,first aid
92,6,76,129683,1,1.908341,17.180353,indian foods
191,11,44,128588,1,1.892228,19.072581,eye ear care
195,12,35,116026,1,1.707372,20.779953,poultry counter
194,12,35,116025,1,1.707358,22.487311,poultry counter


In [100]:
grouped_dd = grouped_dd.drop('cumulative %', axis = 1)

In [108]:
grouped_by_aisle = grouped_dd.groupby('aisle').agg(sum).compute()
# def identity(x):
#     return x

# grouped_by_aisle = grouped_dd.groupby('aisle').agg(
#     {'department_id': identity, 'aisle_id': identity,
#      'counts': sum, 'reordered': identity, '%': sum}).compute()

In [109]:
print(grouped_by_aisle.shape)

(134, 5)


In [110]:
grouped_by_aisle = grouped_by_aisle.nlargest(n=300, columns='%')

In [111]:
pd.options.display.max_rows = 999

In [112]:
grouped_by_aisle

,department_id,aisle_id,counts,reordered,%
aisle,,,,,
baby bath body care,36,204,393405,2,5.789124
packaged produce,8,64,366323,2,5.390600
indian foods,12,152,274685,2,4.042108
first aid,22,236,239636,2,3.526347
poultry counter,24,70,232051,2,3.414730
eye ear care,22,88,226019,2,3.325967
frozen juice,2,226,206356,2,3.036617
seafood counter,24,78,194793,2,2.866463
eggs,32,172,189948,2,2.795167


## So far, we have seen which type of products are sold more than others. We can aggregate all products according to their type (food, personal care products, ...), but it looks like just by looking at it, food products are consumed more. Next we will see how to group these programmatically.